In [ ]:
import random
import csv
import time
import os
from deap import base, creator, tools
from codecarbon import EmissionsTracker


# Limpiar el archivo `emissions.csv` antes de la ejecución
def limpiar_emissions_csv():
    if os.path.exists("emissions.csv"):
        os.remove("emissions.csv")
    print("Archivo emissions.csv eliminado si existía")

limpiar_emissions_csv()

def safe_int(value):
    try:
        return int(value) if value else 0
    except ValueError:
        return 0

def safe_float(value):
    try:
        return float(value) if value else 0.0
    except ValueError:
        return 0.0

# Función de evaluación (OneMax)
def evaluate(individual):
    return sum(individual),

# Configuración de los parámetros
poblaciones = [2**6, 2**10, 2**14]
cruces = [0.2, 0.01, 0.8]
mutaciones = [0.1]
tamaño_individuo = 100  # Ahora el tamaño de los individuos es 100

# Configuración inicial para DEAP
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# Ejecución con DEAP
for poblacion in poblaciones:
    for cruce in cruces:
        for mutacion in mutaciones:
            for ejecucion_num in range(10):  # Ejecutar cada combinación 10 veces
                print(f'Ejecutando combinación {poblacion}, {cruce}, {mutacion} - ejecución {ejecucion_num + 1}')

                # Configurar y comenzar el rastreador de emisiones (asegurando que guarde un CSV)
                tracker = EmissionsTracker(
                    output_dir=".",  # Directorio actual
                    output_file="emissions.csv",  # Nombre del archivo explícito
                    save_to_file=True  # Asegurarse de que guarde a un archivo
                )
                tracker.start()
                print("Tracker de emisiones iniciado")

                # Configuración del toolbox para esta ejecución
                toolbox = base.Toolbox()
                toolbox.register("attr_bool", random.randint, 0, 1)
                toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, tamaño_individuo)
                toolbox.register("population", tools.initRepeat, list, toolbox.individual)
                
                toolbox.register("evaluate", evaluate)
                toolbox.register("mate", tools.cxTwoPoint)
                toolbox.register("mutate", tools.mutFlipBit, indpb=mutacion)
                toolbox.register("select", tools.selTournament, tournsize=3)
                
                # Crear población inicial
                population = toolbox.population(n=poblacion)
                
                # Ejecutar el algoritmo evolutivo
                start_time = time.time()
                best_individual = None
                prev_fitness = 0  # Para calcular la variación entre generaciones
                fitness_inicial = 0  # Inicializar fitness inicial
                
                # Evaluar la población inicial para obtener el fitness inicial
                fitnesses = map(toolbox.evaluate, population)
                for ind, fit in zip(population, fitnesses):
                    ind.fitness.values = fit
                
                best_individual = tools.selBest(population, 1)[0]
                fitness_inicial = best_individual.fitness.values[0]
                prev_fitness = fitness_inicial
                
                # Inicializamos consumo_generacion
                consumo_generacion = 0.0
                
                for generation in range(1000000):  # No limitamos el número de generaciones
                    # Medir el consumo de la generación
                    gen_start_time = time.time()

                    # Realizar cruce y mutación
                    offspring = list(map(toolbox.clone, population))
                    for child1, child2 in zip(offspring[::2], offspring[1::2]):
                        if random.random() < cruce:
                            toolbox.mate(child1, child2)
                            del child1.fitness.values
                            del child2.fitness.values
                    for mutant in offspring:
                        if random.random() < mutacion:
                            toolbox.mutate(mutant)
                            del mutant.fitness.values
                    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
                    fitnesses = map(toolbox.evaluate, invalid_ind)
                    for ind, fit in zip(invalid_ind, fitnesses):
                        ind.fitness.values = fit
                    population[:] = toolbox.select(offspring, len(population))

                    # Obtener el mejor individuo
                    best_individual = tools.selBest(population, 1)[0]
                    fitness_generacion = best_individual.fitness.values[0]

                    # Calcular la variación de fitness
                    variacion_fitness = fitness_generacion - prev_fitness
                    prev_fitness = fitness_generacion

                    # Obtener el tiempo de la generación
                    tiempo_generacion = time.time() - gen_start_time

                    # Verificar si se ha alcanzado el fitness 100% o el tiempo máximo de 2 minutos
                    if fitness_generacion == tamaño_individuo or time.time() - start_time >= 120:
                        fitness_final = fitness_generacion
                        break

                # Parar el seguimiento de consumo energético y guardar resultados
                emissions_value = tracker.stop()
                print(f"Tracker detenido. Emisiones finales: {emissions_value}")
                
                # Intentar guardar explícitamente (por si acaso)
                try:
                    tracker.save()
                    print("Emisiones guardadas explícitamente con tracker.save()")
                except Exception as e:
                    print(f"Error al guardar emisiones: {e}")
                
                # Mostrar resultados en consola
                tiempo_total = time.time() - start_time
                print(f'Resultados de la ejecución {ejecucion_num + 1}:')
                print(f'  Framework: DEAP')
                print(f'  Población: {poblacion}')
                print(f'  Cruce: {cruce}')
                print(f'  Mutación: {mutacion}')
                print(f'  Generación final: {generation}')
                print(f'  Fitness inicial: {fitness_inicial}')
                print(f'  Variación fitness: {variacion_fitness}')
                print(f'  Fitness final: {fitness_final}')
                print(f'  Tiempo total: {tiempo_total} segundos')
                print(f'  Emisiones: {emissions_value}')
                
                # Intentar cargar el archivo CSV si existe para mostrar información adicional
                csv_file_path = "emissions.csv"
                if os.path.exists(csv_file_path):
                    print(f"Archivo CSV encontrado: {csv_file_path}")
                    try:
                        with open(csv_file_path, mode='r') as file:
                            reader = csv.DictReader(file)
                            last_row = None
                            for row in reader:
                                last_row = row
                            if last_row is not None:
                                print("Datos adicionales de CodeCarbon:")
                                print(f"  Duración: {last_row.get('duration', 'N/A')} segundos")
                                print(f"  Energía consumida: {last_row.get('energy_consumed', 'N/A')} kWh")
                                print(f"  Emisiones CO2: {last_row.get('emissions', 'N/A')} kg")
                    except Exception as e:
                        print(f"Error al leer o procesar el archivo CSV: {e}")
                else:
                    print(f"ADVERTENCIA: Archivo CSV no encontrado en {csv_file_path}")

                print(f'Ejecución {ejecucion_num + 1} terminada: Población {poblacion}, Cruce {cruce}, Mutación {mutacion}, Generación {generation}, Fitness final {fitness_generacion}, Tiempo {tiempo_total} segundos, Emisiones: {emissions_value}')
                print("-" * 80)

print("Programa finalizado.")